In [1]:
import hanabi_multiagent_framework as hmf
from hanabi_multiagent_framework.utils import make_hanabi_env_config
import numpy as np
from numpy import ndarray
from hanabi_agents.rule_based import ParallelRulebasedAgent
import hanabi_agents.rule_based.predefined_rules as rules
from hanabi_learning_environment import pyhanabi, rl_env
from hanabi_agents.rule_based.Diversity import Diversity
from hanabi_agents.rule_based.ruleset import Ruleset 
from evolve import Evolution
import timeit
import statistics
import dill as pickle
import math

## Intra-Agent Entropy Code

In [189]:
def Intra_Agent_Entropy(my_rules):

    names_dict = {}

    for i in rule_names:
        names_dict[i] = 1e-6 

    agent_rules = []
    #For all agents, make a names dictionary
    for _ in range(my_rules.shape[0]):
        agent_rules.append(names_dict.copy())


    agent_rules = np.array(agent_rules) 

    for i in range(my_rules.shape[0]):
        rule_list = my_rules[i]
        agent = agent_rules[i]
        for rule in rule_list:
            if agent[rule.__name__] == 1e-6:
                agent[rule.__name__] = 1

            else:
                agent[rule.__name__] += 1

    # Update Probabilities
    agent_probs = []

    for agent in agent_rules:
        agent_probs.append(agent.copy())

    for agent in agent_probs:
        for rule in agent.keys():
            agent[rule] = agent[rule] / my_rules.shape[1]


    # Calculate Entropy
    Entropies = []
    for agent in agent_probs:
        E = 0
        for rules in agent.keys():
            E += - agent[rule] * math.log(agent[rule])

        Entropies.append(E)

    Entropies = np.array(Entropies)

    return Entropies

In [207]:
population_size = 5
n_rules = 20
my_rules = np.array([rules.random_rules(n_rules) for _ in range(population_size)])

# diversity = Intra_Agent_Entropy(my_rules)

In [226]:
rule_names = []
for i in range(my_rules.shape[0]):
    for j in range(my_rules.shape[1]):
        rule_names.append(my_rules[i,j].__name__)

rule_names = np.unique(rule_names)

(12,)

array(['discard_oldest_first', 'discard_randomly', 'hail_mary',
       'osawa_discard', 'play_probably_safe_treshold',
       'play_probably_useless_treshold', 'tell_anyone_useful_card',
       'tell_anyone_useless_card', 'tell_dispensable',
       'tell_most_information', 'tell_playable_card_outer',
       'tell_randomly'], dtype='<U30')

In [212]:
diversity

array([2, 3, 0, 1, 4])

In [215]:
rule_names

array(['discard_oldest_first', 'discard_randomly', 'hail_mary',
       'osawa_discard', 'play_probably_safe_treshold',
       'play_probably_useless_treshold', 'tell_anyone_useful_card',
       'tell_anyone_useless_card', 'tell_dispensable',
       'tell_most_information', 'tell_playable_card_outer',
       'tell_randomly'], dtype='<U30')

## Categorical Name Distance

In [2]:
population_size = 5
n_rules = 20

my_rules = []

for _ in range(population_size):
    x, _ = rules.random_rules(n_rules) 
    my_rules.append(np.array(x))

my_rules = np.array(my_rules)

In [3]:
my_rules.shape

(5, 20)

In [4]:
A = np.array(my_rules[0])
B = np.array(my_rules[3])

In [5]:
rulebase = np.array(rules.big_ruleset)

axes = [] 
weights = []

for rule in rulebase:     
    
    if rule.__name__ in axes:
            weights[axes.index(rule.__name__)] += 1
    else:
        axes.append(rule.__name__)
        weights.append(1)


        

axes = np.array(axes)
weights = np.array(weights)

In [6]:
weights

array([10,  5,  8,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  4,  1,  4])

In [7]:
axes

array(['play_probably_safe_treshold', 'play_probably_useless_treshold',
       'tell_dispensable', 'hail_mary', 'tell_playable_card_outer',
       'tell_randomly', 'tell_anyone_useful_card',
       'tell_anyone_useless_card', 'tell_most_information',
       'discard_randomly', 'osawa_discard', 'discard_oldest_first',
       'tell_if_certain_playable', 'discard_least_playable',
       'play_new_hinted', 'tell_safe_new'], dtype='<U30')

In [8]:
A_ = np.zeros(axes.shape[0])
B_ = np.zeros(axes.shape[0])

for rule in A: 
    index = np.where(axes == rule.__name__)
    A_[index] += 1
    
for rule in B: 
    index = np.where(axes == rule.__name__ )
    B_[index] += 1

In [9]:
# A_ = A_ / weights
A_

array([6., 2., 6., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 4., 0., 0.])

In [10]:
# B_ = B_ / weights
B_

array([3., 2., 4., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 3., 1., 2.])

In [50]:
np.linalg.norm(A_ - B_)

1.7392083483620777

In [14]:
1 - (A_ @ B_)/ (np.linalg.norm(A_) * np.linalg.norm(B_))


0.12165028209126483

In [13]:
Diversity.Name_Distance(A, B, rulebase)

3.1622776601683795

In [82]:
score = A_  @ B_

sqr = np.mean(A_) * np.mean(B_) * A_.shape[0] * B_.shape[0]

# np.sqrt(sqr)

# score * 1 / np.sqrt(sqr)

1 - score / np.sqrt(sqr)


-3.7

In [75]:
def bhattacharya (a,  b):
    
    assert(a.shape[0] == b.shape[0])
    
    
    # calculate means
    a_ = np.mean(a)
    b_ = np.mean(b)

    # calculate score
    score = a @ b
    
    score = np.sqrt( 1 - ( 1 / np.sqrt(a_*b_*a.shape[0]*b.shape[0]) ) * score )
    
    return score

In [76]:
bhattacharya(A_, B_)

<ipython-input-75-7a1042abe2f4>:13: RuntimeWarning: invalid value encountered in sqrt
  score = np.sqrt( 1 - ( 1 / np.sqrt(a_*b_*a.shape[0]*b.shape[0]) ) * score )


nan

## Checking the Evaluation procedure

In [6]:
population_size = 5
n_rules = 20
n_parallel = 50 
n_players = 2
n_generations = 2
n_games_per_pair = 10 


# Initialize the environment
env_conf = make_hanabi_env_config('Hanabi-Full', n_players)
env = hmf.HanabiParallelEnvironment(env_conf, n_parallel)


my_rules = []

for _ in range(population_size):
    x, _ = rules.random_rules(n_rules) 
    my_rules.append(np.array(x))

my_rules = np.array(my_rules)


scores = np.zeros((population_size, population_size, n_games_per_pair))

        # individual_scores = []

        # Agent plays against all the other agents n_parallel times / population_size
        # But this happens sequentially -> TODO: Parallelize this loop
for k in range(population_size):
    # For each agent in the population, create a second agent composed of all other agents
    agents = [ParallelRulebasedAgent(
        [my_rules[k]], n_parallel), ParallelRulebasedAgent(my_rules, n_parallel)]

    # Start a parallel session with these two agents and pass the number of games to be played between
    # them as the environment config
    parallel_session = hmf.HanabiParallelSession(env, agents)

    # Get the result and append the scores
    result = np.array(parallel_session.run_eval(dest=None, print_intermediate=False))
    # print(f"Shape of Result ----> {np.array(result).shape} ")

    result = np.array(result)
    # len(result) = {population_size * n_games } = 200 == 20 * 10

    # split the results into a 2D array of individual scores
    scores[k] = np.array(np.split(result, population_size))

scores

array([[[ 5.,  7.,  1.,  4.,  7., 13.,  6., 15.,  3.,  7.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 4.,  9.,  4.,  9.,  0.,  2.,  8.,  6.,  8.,  4.],
        [ 8.,  0.,  9., 15., 12.,  7., 10.,  7.,  0., 11.],
        [ 9., 12., 11.,  4.,  3.,  5., 11., 14., 13.,  5.]],

       [[ 0.,  0., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  5.,  4.,  3.,  0.,  0.,  3.]],

       [[ 7.,  2.,  7.,  5.,  7.,  3.,  3.,  6.,  4.,  6.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 4.,  7.,  5.,  4.,  5.,  8.,  5.,  1.,  4.,  3.],
        [ 1.,  3.,  1.,  7.,  1.,  1.,  3.,  5.,  3., 11.],
        [14.,  3.,  0., 11.,  4.,  3.,  9.,  0.,  2.,  6.]],

       [[ 6.,  9., 10.,  7.,  9., 14.,  9.,  7., 10., 12.],
        [ 0.,  0.,  0.,  0.,  0., 

In [28]:
# scores.shape

a = np.mean(scores, axis = 2)
a

array([[6.8, 0. , 5.4, 7.9, 8.7],
       [1.5, 0. , 0. , 0. , 1.8],
       [5. , 0. , 4.6, 3.6, 5.2],
       [9.3, 0. , 7.9, 5.5, 6.6],
       [7.8, 2.8, 5.7, 9.2, 6.4]])

In [29]:
np.sort(a, axis =1)

array([[0. , 5.4, 6.8, 7.9, 8.7],
       [0. , 0. , 0. , 1.5, 1.8],
       [0. , 3.6, 4.6, 5. , 5.2],
       [0. , 5.5, 6.6, 7.9, 9.3],
       [2.8, 5.7, 6.4, 7.8, 9.2]])

In [ ]:
x = np.mean()

In [4]:
x = np.mean(individual_scores, axis = 2)

In [5]:
scores

array([[ 0. ,  0. ,  0. ,  7.3,  2.6],
       [ 1.7,  0. ,  0. ,  2.3,  0. ],
       [ 0. ,  0. ,  1.6,  7.3,  2.6],
       [ 6. ,  1.1,  2.5, 11.5,  9.2],
       [ 3.6,  0. ,  2.6,  2.3,  1.5]])

In [6]:
x

array([[ 0. ,  0. ,  0. ,  7.3,  2.6],
       [ 1.7,  0. ,  0. ,  2.3,  0. ],
       [ 0. ,  0. ,  1.6,  7.3,  2.6],
       [ 6. ,  1.1,  2.5, 11.5,  9.2],
       [ 3.6,  0. ,  2.6,  2.3,  1.5]])

In [64]:
np.mean(x, axis = 1)

array([0.58, 2.18, 2.84, 0.68, 0.5 ])

In [67]:
np.mean([0. , 7.3, 1.4, 1.6, 0.6]) 

2.1799999999999997

In [76]:
alpha = 0.01
beta = 0.2
 
performances = np.zeros((x.shape[0], x.shape[1]))

for i in range(x.shape[0]): 
    for j in range(x.shape[1]):
        d_i_j = x[i,j] + alpha * Diversity.Name_Distance(my_rules[i], my_rules[j], rulebase) 
        performances[i,j] = beta * d_i_j

    
        


In [77]:
performances = np.mean(performances, axis = 1)

performances

array([0.12145005, 0.44247924, 0.57272117, 0.14107313, 0.1047392 ])

In [78]:
np.argsort(performances)

array([4, 0, 3, 1, 2])

In [7]:
fitness = np.mean(np.sort(x, axis =1)[-2:], axis =1)

In [21]:
y = np.sort(x, axis = 1 )

In [22]:
y

array([[ 0. ,  0. ,  0. ,  2.6,  7.3],
       [ 0. ,  0. ,  0. ,  1.7,  2.3],
       [ 0. ,  0. ,  1.6,  2.6,  7.3],
       [ 1.1,  2.5,  6. ,  9.2, 11.5],
       [ 0. ,  1.5,  2.3,  2.6,  3.6]])

In [23]:
z = [ arr[-2:] for arr in y]

In [24]:
z = np.

[array([2.6, 7.3]),
 array([1.7, 2.3]),
 array([2.6, 7.3]),
 array([ 9.2, 11.5]),
 array([2.6, 3.6])]

In [25]:
z = np.mean(z, axis=1)

In [26]:
z

array([ 4.95,  2.  ,  4.95, 10.35,  3.1 ])

In [39]:
ls = np.array( [100,600,900,1200,1500] )

ls / np.linalg.norm(ls) 

array([0.04531433, 0.27188595, 0.40782893, 0.54377191, 0.67971488])

In [36]:
s = 0

for k in ls_2:
    s += k 
    
s 

55

In [37]:
np.sqrt(s)

7.416198487095663

In [57]:
np.linalg.norm ( [0, 0.7] + [0.7, 0] )

0.9899494936611665

In [64]:
a = np.array([1,2,3,4])
b = np.array([5,6,7,8])

a @ b

70

In [65]:
5 + 12 + 21 + 32

70

In [14]:
p = np.dstack(([[1,2], [2,3]], [[2,3], [3,4]] ))

In [22]:
a = np.array(
    [
        [1,2,3],
        [4,5,6]
    ]
)

b = np.array(
    [
        [4,5,6],
        [7,8,9]
    ]
)

a.shape

(2, 3)

In [21]:
p = np.dstack((a,b))
p.shape

(2, 3, 2)

In [23]:
p = np.dstack((p, a))
p.shape

(2, 3, 3)

In [24]:
p 

array([[[1, 4, 1],
        [2, 5, 2],
        [3, 6, 3]],

       [[4, 7, 4],
        [5, 8, 5],
        [6, 9, 6]]])